# Logistic Regression Optuna Optimizer

## For Kaggle AMEX default prediction competition data

https://www.kaggle.com/competitions/amex-default-prediction/data

This competition had some random looking data where no information on columns was given. The notebook is mainly to allow me to copy it for use in Logreg optimization in other tabular data cases.

This assumes skelearn and various other Python libraries are installed. This one is not using GPU (as far as I know :)). I used a Docker image and Pipfile with all these installed.


In [2]:
import pandas as pd
import numpy as np

import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import gc
from sklearn import preprocessing

tqdm.pandas()

import time

N_FOLDS = 10

In [3]:
# https://www.kaggle.com/kyakovlev
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)


In [4]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'] 


In [5]:
def format_time(seconds):
    seconds = int(seconds)
    minutes = seconds // 60
    hours = minutes // 60
    minutes = minutes % 60
    seconds = seconds % 60
    if hours > 0:
        return f"{hours}h, {minutes}m, {seconds}s"
    if minutes > 0:
        return f"{minutes}m, {seconds}s"
    return f"{seconds}s"
        

In [6]:
def find_new_cat_cols(df_from):
    new_cat_cols = []
    for cat_col in cat_cols:
        new_cat_cols.extend([col for col in df_from.columns if col.startswith(cat_col)])
    return new_cat_cols


# LogReg Optuna Optimizer

In [82]:

optuna_eval_results = {}
optuna_iteration_index = 0
optuna_losses = []
optuna_accuracies = []
optuna_amexes = []
optuna_parameters = []
optuna_seconds = []

def optuna_objective(trial, X, y, verbosity=-1):
    global optuna_iteration_index
    optuna_iteration_index += 1
    param_grid = {
        "solver": trial.suggest_categorical("solver", ["lbfgs", "liblinear", "sag", "saga"]),
        'C': trial.suggest_discrete_uniform('C', 1e-5, 10, 0.1),
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        "class_weight": trial.suggest_categorical("class_weight", ["balanced", None]),
        "n_jobs": -1,
        "verbose": 0
    }
    if param_grid["solver"] == "lbfgs":
        penalty = trial.suggest_categorical("penalty-lbfgs", ["l2", 'none'])
        param_grid["penalty"] = penalty
        print(f"1: {penalty}")
    if param_grid["solver"] == "liblinear":
        penalty = trial.suggest_categorical("penalty-liblinear", ["l1", "l2"])
        param_grid["penalty"] = penalty
        print(f"2: {penalty}")
    if param_grid["solver"] == "sag":
        penalty = trial.suggest_categorical("penalty-sag", ["l2", 'none'])
        param_grid["penalty"] = penalty
        print(f"3: {penalty}")
    if param_grid["solver"] == "saga":
        penalty = trial.suggest_categorical("penalty-saga", ["elasticnet", "l1", "l2", 'none'])
        param_grid["penalty"] = penalty
        print(f"4: {penalty}")
    if param_grid["penalty"] == "elasticnet":
        param_grid["l1_ratio"] = trial.suggest_discrete_uniform('l1_ratio', 0.00001, 0.99999, 0.00001)
        print(f"5: {param_grid['penalty']}")
    
    print(f"STARTING OPTIMIZATION iteration {optuna_iteration_index}, data size: {X.shape}, params={param_grid}")
    #split data into 5 pieces, run the optimizer on only 2 of them for speed
    N_FOLDS_RUN = N_FOLDS
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=69)
#    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1121218)

    cv_scores = np.empty(N_FOLDS_RUN)
    cv_accuracies = np.empty(N_FOLDS_RUN)
    cv_amexes = np.empty(N_FOLDS_RUN)
    
    time_start_all_folds = time.time()
    acc_score_total = 0
    n_classes = 1
    train_preds = np.zeros((X.shape[0], n_classes))
    amex_total = 0
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        if idx >= N_FOLDS_RUN:
            break
        time_start_this_fold = time.time()
        print(f"=== STARTING FOLD {idx+1}/{N_FOLDS_RUN} ({N_FOLDS}) ===")
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
#        evalset = [(X_train, y_train), (X_test,y_test)]
        evalset = [(X_test,y_test)]
        
        print("creating classifier")
        model = LogisticRegression(**param_grid)
        print("fitting")
                
        model.fit(
            X_train,
            y_train,
        )

        print("predicting")
        preds = model.predict_proba(X_test)

        preds_true_amex = preds[:, 1]
        preds_true = preds[:, 1].flatten()
        preds_true.shape = (y_test.shape[0], 1)
        amex_y_test = np.array(y_test)
        amex_y_test.shape = (y_test.shape[0], 1)
        np.add.at(train_preds, test_idx, preds_true)
        print(f"count zerO: {np.count_nonzero(train_preds)}")

        preds_this_round = (preds >= 0.5)[:,1]

        acc_score = accuracy_score(y_test, preds_this_round)
        acc_score_total += acc_score

        amex_score = amex_metric_mod(y_test, preds_true_amex)
        amex_total += amex_score

        log_loss_fold = log_loss(y_test, preds)
        cv_scores[idx] = log_loss_fold
        cv_accuracies[idx] = acc_score
        cv_amexes[idx] = amex_score
        print(f"amex: {amex_score}")

        print(f"=== FINISHED FOLD {idx+1}/{N_FOLDS}, log loss={log_loss_fold}, accuracy={acc_score}, amex={amex_score} ===")
        time_total_this_fold = time.time() - time_start_this_fold
        print(f"time to run this fold: {format_time(time_total_this_fold)}")        

        if amex_score < 0.78:
            print("low score on amex, dropping loop")
            return amex_score

   
    time_total_all_folds = time.time() - time_start_all_folds        
    print(f"time to run folds: {format_time(time_total_all_folds)}")
    
    iteration_log_loss = np.mean(cv_scores)
    iteration_accuracy = np.mean(cv_accuracies)
    iteration_amex = np.mean(cv_amexes)
    optuna_losses.append(iteration_log_loss)
    optuna_amexes.append(iteration_amex)
    optuna_accuracies.append(iteration_accuracy)
    optuna_parameters.append(param_grid)
    optuna_seconds.append(int(time_total_all_folds))
    print(f"iteration logloss: {iteration_log_loss}, iteration acc: {iteration_accuracy}, Iteration amex: {iteration_amex}")
    return iteration_amex #iteration_log_loss


# Load Data

The data is the prediction outputs of XGB, LGBM, Catboost, CNN1D, and CNN Wavenet models.

## First the predictions for the training data, to be used to train the logreg ensemble model:

In [83]:
df_xgb = pd.read_csv("predictions_xgb_lagged.csv").drop("Unnamed: 0", axis=1)
df_lgbm = pd.read_csv("predictions_lgbm_lagged.csv").drop("Unnamed: 0", axis=1)
df_cat = pd.read_csv("predictions_cat_lagged.csv").drop("Unnamed: 0", axis=1)
df_cnn = pd.read_csv("predictions_cnn.csv").drop("Unnamed: 0", axis=1)
df_wave = pd.read_csv("predictions_wave.csv").drop("Unnamed: 0", axis=1)
df_target = pd.read_csv("train_labels.csv")

In [84]:
df_xgb

,customer_ID,prediction
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.000736
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.001333
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.001681
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.007314
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.001709
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.002823
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.023796
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.003624
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.055756


In [85]:
#these tuples will be later merged into one, such as prediction_xgb, prediction_cat, ... 
#where the first tuple item (tupe[0]) is the prediction dataframe, and tuple[1] is its prefix
dfs_to_merge = [(df_xgb, "xgb"), (df_cat, "cat"), (df_cnn, "cnn"), (df_wave, "wave"), (df_target, "target")]

In [86]:
df_train = pd.DataFrame()
df_train["customer_ID"] = []
#tuple0 = dfs_to_merge[0]
#df_train[f"prediction_{tuple0[1]}"] = tuple0[0]["prediction"]

for df_tuple in dfs_to_merge:
    df = df_tuple[0]
    suffix = df_tuple[1]
    print(f"merging {suffix}")
    df_train = pd.merge(df_train, df, how="outer", on="customer_ID", suffixes=["", f"_{suffix}"])


merging xgb
merging cat
merging cnn
merging wave
merging target


In [87]:
df_train

,customer_ID,prediction,prediction_cat,prediction_cnn,prediction_wave,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.000736,0.001574,0.000114,0.000582,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.001333,0.002910,0.000693,0.000715,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.001681,0.002506,0.000670,0.001487,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.007314,0.007465,0.003867,0.003853,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.001709,0.005253,0.001066,0.002279,0
...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.002823,0.007015,0.005916,0.004779,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.023796,0.024286,0.021221,0.013617,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.003624,0.002882,0.000570,0.001506,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.055756,0.054378,0.112111,0.053124,1


## The Input Submission Predictions to Predict Final Submission

In [112]:
df_xgb = pd.read_csv("submission_xgb_lagged.csv")
df_lgbm = pd.read_csv("submission_lgbm_lagged_seed2.csv")
df_cat = pd.read_csv("submission_cat_lagged_seed2.csv")
df_cnn = pd.read_csv("submission_cnn.csv")
df_wave = pd.read_csv("submission_cnn_wave.csv")

#df_train = df_xgb.copy()

In [113]:
dfs_to_merge = [(df_xgb, "xgb"), (df_cat, "cat"), (df_cnn, "cnn"), (df_wave, "wave")]

In [114]:
df_test = pd.DataFrame()
df_test["customer_ID"] = []

for df_tuple in dfs_to_merge:
    df = df_tuple[0]
    suffix = df_tuple[1]
    print(f"merging {suffix}")
    df_test = pd.merge(df_test, df, how="outer", on="customer_ID", suffixes=["", f"_{suffix}"])


merging xgb
merging cat
merging cnn
merging wave


# Train and Predict After Optimization

In [103]:
def train_and_predict(optuna_params, df_from):
    X = df_from.drop(["customer_ID", "target"], axis=1)
    y = df_from["target"]

    verbosity=50 #print status every 50 iteration of tree building
    optuna_eval_results = {}

    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1121218)

    cv_scores = np.empty(N_FOLDS)
    cv_accuracies = np.empty(N_FOLDS)
    time_start_all_folds = time.time()
    acc_score_total = 0
    models = []

    n_classes = 1
    train_preds = np.zeros((X.shape[0], n_classes))
    #balanced_preds = np.empty([X.shape[0], n_classes], dtype = float)

    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):

        
        time_start_this_fold = time.time()
        print(f"=== STARTING FOLD {idx+1}/{N_FOLDS} ===")
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        print("creating classifier")
        model = LogisticRegression(**optuna_params)

        
        print("fitting")
        model.fit(
            X_train,
            y_train,
        )
        
        print(f"predicting")
        preds = model.predict_proba(X_test)

        preds_true_amex = preds[:, 1]
        preds_true = preds[:, 1].flatten()
        preds_true.shape = (y_test.shape[0], 1)
        np.add.at(train_preds, test_idx, preds_true)
        print(f"count zerO: {np.count_nonzero(train_preds)}")
        
        amex_score = amex_metric_mod(y_test, preds_true_amex)        

        preds_this_round = (preds >= 0.5)[:,1]
        
        acc_score = accuracy_score(y_test, preds_this_round)
        acc_score_total += acc_score

        log_loss_fold = log_loss(y_test, preds)
        cv_scores[idx] = log_loss_fold
        cv_accuracies[idx] = acc_score
        print(f"=== FINISHED FOLD {idx+1}/{N_FOLDS}, log loss={log_loss_fold}, accuracy={acc_score}, amex={amex_score} ===")
        time_total_this_fold = time.time() - time_start_this_fold
        print(f"time to run this fold: {format_time(time_total_this_fold)}")        
        
        models.append(model)
        
    return models, train_preds


In [89]:
#df_train = pd.read_parquet("train_lagged.parquet", engine="pyarrow")
df_train.head()

,customer_ID,prediction,prediction_cat,prediction_cnn,prediction_wave,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.000736,0.001574,0.000114,0.000582,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.001333,0.002910,0.000693,0.000715,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.001681,0.002506,0.000670,0.001487,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.007314,0.007465,0.003867,0.003853,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.001709,0.005253,0.001066,0.002279,0


In [90]:
#df_test = pd.read_parquet("test_lagged.parquet", engine="pyarrow")
df_test.head()

NameError: name 'df_test' is not defined

In [91]:
X = df_train.drop(["customer_ID", "target"], axis=1)
y = df_train["target"]

# Run Actual Optuna Optimization for Logreg on Training Data

In [92]:
optuna_eval_results = {}
optuna_iteration_index = 0
optuna_losses = []
optuna_accuracies = []
optuna_amexes = []
optuna_parameters = []
optuna_seconds = []

#study = optuna.create_study(direction="minimize", study_name="XGB Classifier")
study = optuna.create_study(direction="maximize", study_name="LogReg Classifier")
func = lambda trial: optuna_objective(trial, X, y)
study.optimize(func, n_trials=50)

[I 2022-08-11 18:20:25,031] A new study created in memory with name: LogReg Classifier
/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [1e-05, 10] and q=0.1, but the range is not divisible by `q`. It will be replaced by [1e-05, 9.90001].
  warnings.warn(


3: l2
STARTING OPTIMIZATION iteration 1, data size: (458913, 4), params={'solver': 'sag', 'C': 7.80001, 'fit_intercept': True, 'class_weight': None, 'n_jobs': -1, 'verbose': 0, 'penalty': 'l2'}
=== STARTING FOLD 1/10 (10) ===
creating classifier
fitting
predicting
count zerO: 45892
amex: 0.790076908516802
=== FINISHED FOLD 1/10, log loss=0.23464963167002814, accuracy=0.9048853830733026, amex=0.790076908516802 ===
time to run this fold: 1s
=== STARTING FOLD 2/10 (10) ===
creating classifier
fitting
predicting
count zerO: 91784
amex: 0.798255622855955
=== FINISHED FOLD 2/10, log loss=0.22987875267991098, accuracy=0.9051468665562625, amex=0.798255622855955 ===
time to run this fold: 1s
=== STARTING FOLD 3/10 (10) ===
creating classifier
fitting
predicting
count zerO: 137676
amex: 0.7972424006199069
=== FINISHED FOLD 3/10, log loss=0.228409411323274, accuracy=0.9070208315174758, amex=0.7972424006199069 ===
time to run this fold: 1s
=== STARTING FOLD 4/10 (10) ===
creating classifier
fittin

[I 2022-08-11 18:20:36,086] Trial 0 finished with value: 0.7933035108781096 and parameters: {'solver': 'sag', 'C': 7.80001, 'fit_intercept': True, 'class_weight': None, 'penalty-sag': 'l2'}. Best is trial 0 with value: 0.7933035108781096.


predicting
count zerO: 458913
amex: 0.7868648077206741
=== FINISHED FOLD 10/10, log loss=0.2367486836547948, accuracy=0.9024863262949162, amex=0.7868648077206741 ===
time to run this fold: 1s
time to run folds: 11s
iteration logloss: 0.23257269671953687, iteration acc: 0.904358769577736, Iteration amex: 0.7933035108781096
4: elasticnet
5: elasticnet
STARTING OPTIMIZATION iteration 2, data size: (458913, 4), params={'solver': 'saga', 'C': 2.00001, 'fit_intercept': True, 'class_weight': 'balanced', 'n_jobs': -1, 'verbose': 0, 'penalty': 'elasticnet', 'l1_ratio': 0.63909}
=== STARTING FOLD 1/10 (10) ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [1e-05, 10] and q=0.1, but the range is not divisible by `q`. It will be replaced by [1e-05, 9.90001].
  warnings.warn(


...

predicting
count zerO: 413022
amex: 0.7960377707250587
=== FINISHED FOLD 9/10, log loss=0.23072163499851597, accuracy=0.9039463075548583, amex=0.7960377707250587 ===
time to run this fold: 0s
=== STARTING FOLD 10/10 (10) ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2022-08-11 18:29:10,193] Trial 48 finished with value: 0.7933035194008177 and parameters: {'solver': 'lbfgs', 'C': 3.2000100000000002, 'fit_intercept': True, 'class_weight': None, 'penalty-lbfgs': 'none'}. Best is trial 17 with value: 0.7933326074912369.


predicting
count zerO: 458913
amex: 0.7868649388743914
=== FINISHED FOLD 10/10, log loss=0.23674875401528378, accuracy=0.9024863262949162, amex=0.7868649388743914 ===
time to run this fold: 0s
time to run folds: 8s
iteration logloss: 0.23257281215119266, iteration acc: 0.9043609486542433, Iteration amex: 0.7933035194008177
3: none
STARTING OPTIMIZATION iteration 50, data size: (458913, 4), params={'solver': 'sag', 'C': 5.40001, 'fit_intercept': False, 'class_weight': None, 'n_jobs': -1, 'verbose': 0, 'penalty': 'none'}
=== STARTING FOLD 1/10 (10) ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [1e-05, 10] and q=0.1, but the range is not divisible by `q`. It will be replaced by [1e-05, 9.90001].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2022-08-11 18:29:11,280] Trial 49 finished with value: 0.7639832266945181 and parameters: {'solver': 'sag', 'C': 5.40001, 'fit_intercept': False, 'class_weight': None, 'penalty-sag': 'none'}. Best is trial 17 with value: 0.7933326074912369.


predicting
count zerO: 45892
amex: 0.7639832266945181
=== FINISHED FOLD 1/10, log loss=0.6507780762548928, accuracy=0.2985923472500654, amex=0.7639832266945181 ===
time to run this fold: 1s
low score on amex, dropping loop


In [93]:
sum(optuna_seconds)

494

# Look into Optimized Parameters

Compare parameters vs achieved loss, accuracy, amex score, and running time:

In [94]:
df_params = pd.DataFrame(optuna_parameters)
df_params["loss"] = optuna_losses
df_params["accuracy"] = optuna_accuracies
df_params["amex"] = optuna_amexes
df_params["time"] = optuna_seconds

#df_params

In [96]:
df_params.to_csv("optimizer_results_logreg.csv")

In [97]:
optuna_parameters

[{'solver': 'sag',
  'C': 7.80001,
  'fit_intercept': True,
  'class_weight': None,
  'n_jobs': -1,
  'verbose': 0,
  'penalty': 'l2'},
 {'solver': 'saga',
  'C': 2.00001,
  'fit_intercept': True,
  'class_weight': 'balanced',
  'n_jobs': -1,
  'verbose': 0,
  'penalty': 'elasticnet',
  'l1_ratio': 0.63909},
 {'solver': 'lbfgs',
  'C': 3.00001,
  'fit_intercept': True,
  'class_weight': None,
  'n_jobs': -1,
  'verbose': 0,
  'penalty': 'l2'},
 {'solver': 'sag',
  'C': 5.50001,
  'fit_intercept': False,
  'class_weight': 'balanced',
  'n_jobs': -1,
  'verbose': 0,
  'penalty': 'l2'},
 {'solver': 'sag',
  'C': 8.10001,
  'fit_intercept': True,
  'class_weight': None,
  'n_jobs': -1,
  'verbose': 0,
  'penalty': 'l2'},
 {'solver': 'sag',
  'C': 4.300009999999999,
  'fit_intercept': True,
  'class_weight': 'balanced',
  'n_jobs': -1,
  'verbose': 0,
  'penalty': 'l2'},
 {'solver': 'saga',
  'C': 8.30001,
  'fit_intercept': True,
  'class_weight': 'balanced',
  'n_jobs': -1,
  'verbose': 0

In [98]:
best_idx = np.argmax(df_params["amex"])
best_idx

14

In [99]:
df_params.sort_values(by="amex", ascending=False).head(5)

,solver,C,fit_intercept,class_weight,n_jobs,verbose,penalty,l1_ratio,loss,accuracy,amex,time
14,liblinear,6.40001,True,None,-1,0,l1,NaN,0.232574,0.904376,0.793333,16
36,liblinear,1.00001,True,None,-1,0,l1,NaN,0.232573,0.904367,0.793324,15
11,liblinear,6.40001,True,None,-1,0,l1,NaN,0.232574,0.904365,0.793324,14
12,liblinear,6.40001,True,None,-1,0,l1,NaN,0.232573,0.904365,0.793320,18
30,liblinear,1.30001,True,None,-1,0,l1,NaN,0.232572,0.904367,0.793316,16


In [100]:
df_params.shape

(43, 12)

# Pick Best Optimizer Parameters and Clean it up

Remove the metadata added above, and any other excess values that might confuse the classifier.

In [101]:
optuna_params = df_params.iloc[best_idx]
optuna_params = dict(optuna_params)


In [102]:
optuna_params

{'solver': 'liblinear',
 'C': 6.40001,
 'fit_intercept': True,
 'class_weight': None,
 'n_jobs': -1,
 'verbose': 0,
 'penalty': 'l1',
 'l1_ratio': nan,
 'loss': 0.2325737959076788,
 'accuracy': 0.9043762019523817,
 'amex': 0.7933326074912369,
 'time': 16}

In [105]:
keys = list(optuna_params.keys())
for param in keys:
    if isinstance(optuna_params[param], (np.number)) and np.isnan(optuna_params[param]):
        print(f"delete: {param}")
        del optuna_params[param]
print("delete loss")
del optuna_params["loss"]
#del optuna_params["Unnamed: 0"]
del optuna_params["accuracy"]
del optuna_params["time"]
del optuna_params["amex"]


delete: l1_ratio
delete loss


# Train the Final Model with Best Parameters

Could probably use more folds here etc if desired, to see if the model performence could still be slighly improved.

In [106]:
#optuna_params = df_params.iloc[best_idx]
#final_models, final_preds = train_and_predict(optuna_params, df_full_train[:50000])
final_models, final_preds = train_and_predict(optuna_params, df_train)


=== STARTING FOLD 1/10 ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn(


predicting
count zerO: 45892
=== FINISHED FOLD 1/10, log loss=0.22997592302199105, accuracy=0.9056262529416892, amex=0.7995776002457089 ===
time to run this fold: 1s
=== STARTING FOLD 2/10 ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn(


predicting
count zerO: 91784
=== FINISHED FOLD 2/10, log loss=0.23866587890432342, accuracy=0.9007234376361893, amex=0.7842778810721497 ===
time to run this fold: 1s
=== STARTING FOLD 3/10 ===
creating classifier
fitting


...

predicting
count zerO: 367131
=== FINISHED FOLD 8/10, log loss=0.228656407516664, accuracy=0.9054716611100215, amex=0.8012668070223616 ===
time to run this fold: 1s
=== STARTING FOLD 9/10 ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn(


predicting
count zerO: 413022
=== FINISHED FOLD 9/10, log loss=0.2314607230777883, accuracy=0.9060818025320869, amex=0.7979308431254946 ===
time to run this fold: 1s
=== STARTING FOLD 10/10 ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn(


predicting
count zerO: 458913
=== FINISHED FOLD 10/10, log loss=0.2318632904685847, accuracy=0.9052101719291364, amex=0.7920845306271835 ===
time to run this fold: 1s


In [119]:
#test.replace([np.inf, -np.inf], np.nan,inplace=True)
X = df_test.reset_index(drop=True).drop(["customer_ID"], axis=1)


In [120]:
X

,prediction,prediction_cat,prediction_cnn,prediction_wave
0,0.036600,0.021390,0.012639,0.011262
1,0.001210,0.003025,0.000541,0.000376
2,0.028329,0.040909,0.023913,0.017206
3,0.286871,0.223804,0.236606,0.196489
4,0.858424,0.846619,0.804460,0.805402
...,...,...,...,...
924616,0.021910,0.013295,0.008139,0.007367
924617,0.834791,0.856650,0.833293,0.835387
924618,0.575642,0.572674,0.577704,0.576724
924619,0.340214,0.338167,0.367028,0.335191


# Run Final Predictions

Using all models trained from the N data folds, averaging their predictions.

In [121]:
from tqdm import tqdm

n_classes = 1
sub_preds = np.zeros((X.shape[0], n_classes))

#tqdm.auto()
tqdm.pandas()

for model in tqdm(final_models): #lqtm
    preds = model.predict_proba(X)
    preds_true = preds[:, 1]
    preds_true.shape = (X.shape[0], 1)
    sub_preds += preds_true
sub_preds /= N_FOLDS


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 72.18it/s]


In [122]:
submission = pd.DataFrame()
submission["customer_ID"] = df_test.reset_index()["customer_ID"]
submission["prediction"] = sub_preds
submission

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.028070
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.024209
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.029470
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.118674
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.899949
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.026403
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.903488
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.586670
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.216163


In [123]:
submission.to_csv("submission_logreg.csv", index=False)

In [ ]:
for idx, model in enumerate(final_models):    
    with open(f'final_model{idx}.pickle','wb') as f:
        pickle.dump(model,f)

In [42]:
isinstance(np.int32(10), (np.number))

True